In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import re

# %matplotlib inline
%matplotlib qt
# sns.set_context('paper', font_scale=1.5)

import os, glob
from pathlib import Path
from scipy import stats, interpolate, signal
from scipy.signal import welch, butter, filtfilt
from nolds import sampen
from scipy.ndimage import gaussian_filter1d

from HelperFcns import *

%load_ext autoreload
%autoreload 2

In [2]:
plt.close('all')

In [3]:
#load poses
path = '/Users/luca/Box/CISPD_Videos_proc/Metadata/'
#load data
df = pd.read_csv(os.path.join(path, 'PosesPat.csv'))
df.index = df['FrameID']
df.head()

,SubjID,Task,cycle,elbR_x,elbR_y,elbR_c,wriR_x,wriR_y,wriR_c,elbL_x,...,midL_c,ringL_x,ringL_y,ringL_c,pinkyL_x,pinkyL_y,pinkyL_c,PersonID,FrameID,NPerson
FrameID,,,,,,,,,,,,,,,,,,,,,
0,1005,FtnL,1,665.729,792.330,0.766920,868.719,836.455,0.802609,786.395,...,0.659051,944.507,532.372,0.632399,944.507,544.356,0.623629,1,0,3
1,1005,FtnL,1,665.653,795.175,0.750911,865.876,836.453,0.808026,789.355,...,0.647635,943.553,532.087,0.596038,942.785,544.374,0.615199,1,1,3
2,1005,FtnL,1,665.679,795.244,0.742167,865.785,836.502,0.809571,792.186,...,0.616394,843.683,413.160,0.822903,861.124,434.393,0.663917,1,2,3
3,1005,FtnL,1,665.627,792.313,0.760414,862.942,836.496,0.815405,792.326,...,0.804164,842.660,412.750,0.849422,858.018,431.180,0.762118,1,3,3
4,1005,FtnL,1,665.620,795.171,0.755462,862.911,836.500,0.817131,795.149,...,0.656123,843.071,412.394,0.805221,860.170,431.824,0.758662,1,4,3


In [7]:
#load scores
# scores = pd.read_csv('/Users/luca/Box/CISPD_Videos_proc/Metadata/Table4.csv')
tasks = ['Wlkg']
scores = pd.read_csv(path+'/Table4.csv')
scores = scores.loc[(scores['SubjID'].isin(df.SubjID.unique())) & (scores.TaskAbb.isin(tasks))]
scores = scores[['SubjID','Visit','TaskAbb','Tremor - Left', 'Tremor - Right', 
                 'Bradykinesia - Right', 'Bradykinesia - Left','Dyskinesia - Left','Dyskinesia - Right']]
scores.rename(columns={'Tremor - Right':'Tremor - R', 'Tremor - Left':'Tremor - L',
                      'Bradykinesia - Right': 'Bradykinesia - R',
                      'Bradykinesia - Left': 'Bradykinesia - L',
                      'Dyskinesia - Left': 'Dyskinesia - L',
                      'Dyskinesia - Right': 'Dyskinesia - R'}, inplace=True)
scores.query('Visit != "1 Month"', inplace=True)
scores['Visit']=scores.Visit.apply(lambda x: x.split()[-1])
for i,j in enumerate(scores.Visit.unique()):
    print(i,j)
    scores.loc[scores.Visit==j, 'Visit'] = i+1
scores = scores.query('Visit ==1 | Visit ==2 | Visit==3')
scores.head(2)

0 0
1 30
2 60
3 90
4 120
5 150


,SubjID,Visit,TaskAbb,Tremor - L,Tremor - R,Bradykinesia - R,Bradykinesia - L,Dyskinesia - L,Dyskinesia - R
223,1005,1,Wlkg,0.0,0.0,0.0,2.0,0.0,0.0
238,1005,2,Wlkg,0.0,0.0,0.0,3.0,0.0,0.0


In [8]:
#distribution of symptoms
plt.close('all')
for task in tasks:
    plt.figure()
    scores.query('TaskAbb==@task').iloc[:,3:].plot(kind='hist', subplots=True)
    plt.suptitle(task)

## Compute features for both sides - TBD